In [3]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize

In [4]:
sidewalks =gpd.read_file('data/shape/Sidewalks.shx')
sidewalks_hazards = gpd.read_file('data/shape/Sidewalk_Hazards.shx')

In [5]:
geolocator = geopy.Nominatim(user_agent='my-application1')
async def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))

    return location.raw['address']

async def get_response(zipcodes):
    return await asyncio.gather(**zipcodes)
def process_sidewalks_hazards():
    data = folium.GeoJson(data=sidewalks_hazards["geometry"])
    coord = { "x" : [], "y" : [] }
    for i in range(len(data.data['features'])):
        
        coord["x"].append(float(data.data['features'][i]['geometry']['coordinates'][0]))
        coord["y"].append(float(data.data['features'][i]['geometry']['coordinates'][1]))    
    
    zipcodes = []
    for index,row in coord.iterrows():
        zipcodes.append(asyncio.ensure_future(get_zipcode(row, geolocator,lat_field='y', lon_field='x')))
    response = asyncio.run(get_response(zipcodes))
    sidewalks_hazards['address'] = response
    sidewalks_hazards['pincode'] = sidewalks_hazards['address'].apply(lambda x: x['postcode'])
    sidewalks_hazards.to_csv("sidewalks_hazards_withaddress.csv")


In [6]:

sidewalks['SWK_WIDTH'] = (sidewalks['SWK_WIDTH'].str.replace(" ",""))
sidewalks['SWK_WIDTH'] = (sidewalks['SWK_WIDTH']).str.replace("O","0")
sidewalks['SWK_WIDTH'] = sidewalks['SWK_WIDTH'].str.replace("l","")
sidewalks['SWK_WIDTH'] = sidewalks['SWK_WIDTH'].str.replace(",",".")
sidewalks['SWK_WIDTH'] = sidewalks['SWK_WIDTH'].astype(float)
sidewalks['SWK_AREA'] = sidewalks['SWK_AREA'].astype(float)

In [7]:
sidewalks.groupby('DISTRICT').agg({'SWK_WIDTH' : 'mean', 'SWK_AREA' : 'mean'})

,SWK_WIDTH,SWK_AREA
DISTRICT,,
ALLSTON/BRIGHTON,6.542724,2381.861342
CHARLESTOWN,8.149412,1895.730481
DOWNTOWN,8.646959,2864.800000
EAST BOSTON,8.810446,2908.683761
HYDE PARK,6.096727,2433.337152
JAMAICA PLAIN,6.240322,2138.124641
NORTH DORCHESTER,6.325244,2096.217554
NORTH END,7.895376,1699.973580
ROXBURY,8.335938,4670.447917


In [9]:
sidewalks.groupby('MATERIAL').agg({'SWK_WIDTH' : 'mean', 'SWK_AREA' : 'mean'})

,SWK_WIDTH,SWK_AREA
MATERIAL,,
BC,6.452790,2563.448661
BL,6.562500,1720.625000
BR,10.235861,2768.239336
CB,11.441635,3337.020913
CC,6.786158,2357.304937
GB,14.615663,4886.590361
OT,4.976271,2478.186441
OTHER,6.888889,4355.518519
UNK,0.305281,1054.359736


In [8]:
import folium
from folium import plugins, Map
sidewalks =gpd.read_file('data/shape/Sidewalks.shx')
sidewalks_hazards = gpd.read_file('data/shape/Sidewalk_Hazards.shx')
map = folium.Map(location=[42.361145, -71.057083],zoom_start = 30)
data = folium.GeoJson(sidewalks['geometry'])
data.add_to(map)
map.save('maps/sidewalks/Sidewalks.html')

In [10]:
csv_file = pd.read_csv("data/preprocessed_sidewalks_hazards.csv")
csv_file.head()
import json
csv_file["road"] = csv_file["address"].apply(lambda x: eval(x).get("road"))
csv_file["county"] = csv_file["address"].apply(lambda x: eval(x).get("county"))
csv_file["neighbourhood"] = csv_file["address"].apply(lambda x: eval(x).get("neighbourhood"))

# base_map = Map(location=[42.361145, -71.057083],zoom_start = 30)
csv_file['x'] = csv_file['x'].astype(float)
csv_file['y'] = csv_file['y'].astype(float)

def color_cluster(index):
    if index == "Fixed pinch point <36” sidewalk width":
        return 'green'
    elif index == "Trip hazards due to tree roots":
        return 'blue'
    elif index == "Trip hazards not due to tree roots": 
        return 'red'
    else:
        return 'yellow'


#hazards= csv_file.sort_values(by=["Type"],ascending=False)

for point in range(0, len(csv_file)):
    folium.CircleMarker([csv_file.iloc[point]['y'], csv_file.iloc[point]['x']],
                        color = color_cluster(csv_file.iloc[point]['Type']), fill = True, 
                        fill_color = color_cluster(csv_file.iloc[point]['Type'])).add_to(map)

map.save('maps/sidewalks/Sidewalks_hazards.html')